<a href="https://colab.research.google.com/github/scfulford/Serengeti-Camera-Traps/blob/master/density_estimations/AI_FieldOfView_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
test_import <-system('python3 -c "import google.colab"', intern=TRUE)
import_status <- attr(test_import,'status')
datadir<-'../data/'
rootdir<-'../'
if (is.null(import_status)) { 
    datadir<-'camera-trap-analysis/data/'
    rootdir<-'camera-trap-analysis/'
    if (!dir.exists('camera-trap-analysis')){
        install.packages("getPass")

        # no error received so we are running in colab
        username <- readline(prompt="enter GitHub username: ")
        password <- getPass::getPass("enter password: ")
        Sys.setenv(git_user =  paste(username, password, sep=":"))
        system('git clone https://$git_user@github.com/ctorney/camera-trap-analysis.git')
        
    }
}
library(dplyr)

In [ ]:
require(stringr)


Loading required package: stringr



Load the data

In [ ]:
# Load and format Bounding Box predictions
preds <- read.table(paste(rootdir, "aide_query/box_predictions_20200506.txt", sep = ""), sep = ",")
names(preds) <- c("filename", "species", "x", "y", "width", "height")
preds <- preds[2:length(preds$filename),]
preds$filename <- as.character(preds$filename)
preds$filename <- substr(preds$filename, nchar(preds$filename)-36, nchar(preds$filename))
preds$filename <- gsub("_TAB", "TAB", preds$filename)

In [ ]:
# Load Image metadata
All_Images <- read.csv(paste(datadir, "manual_counts.csv", sep = ""))
# New file name column
All_Images$filename <- paste(All_Images$camera.no, All_Images$datedownload, All_Images$folder, All_Images$basename, sep = "_")
# Select only relevant columns
All_Images <- All_Images[,c(1, 2, 4, 5, 55)]
names(All_Images) <- c("basename", "date", "camera.no", "datedownload", "filename")
All_Images$joinid <- paste(All_Images$camera.no, All_Images$datedownload, sep = " ")


In [ ]:
# Load resolution data frame
resolution <- read.csv(paste(datadir, "image_resolution.csv", sep = ""))
resolution$joinid <- paste(resolution$camera.no, resolution$datedownload, sep = " ")


Join Dataframes

In [ ]:
All_Images <- left_join(All_Images, resolution, by = "joinid")
All_Images <- All_Images[,c(1:5, 9)]
names(All_Images)[3:4] <- c("camera.no", "datedownload")
preds <- left_join(preds, All_Images, by = "filename")


In [ ]:
# Change BB dimensions to number objects
preds$x <- as.numeric(as.character(preds$x))
preds$y <- as.numeric(as.character(preds$y))
preds$width <- as.numeric(as.character(preds$width))
preds$height <- as.numeric(as.character(preds$height))

In [ ]:
# Create columns of image dimensions
preds$image_width <- as.numeric(substr(preds$resolution , 1, 4))
preds$image_height <- as.numeric(substr(preds$resolution , 8, 11))


Calculate BB dimensions in pixels


In [ ]:
preds$x_pixel <- preds$x * preds$image_width
preds$y_pixel <- preds$y * preds$image_height
preds$y_pixel <- preds$image_height - preds$y_pixel # Change to pixel count from bottom of image
preds$width_pixel <- preds$width * preds$image_width
preds$height_pixel <- preds$height * preds$image_height
preds <- preds[order(preds$camera.no),]


In [ ]:
summary(preds$camera.no)

KILE_5 KILE_6 KUKA_1 KUKA_2 KUKA_3 KUKA_4 KUKA_5 KUKA_6 MBAL_1 MBAL_2 MBAL_3 
   138     46    119    629    656   1237    543   2525      6   1518     35 
MBAL_4 MBAL_5 MBAL_6 SOIT_1 SOIT_2 SOIT_3 SOIT_4 SOIT_5 SOIT_6  TAB_1  TAB_2 
   247    441   3464     63     34     16     14      0      0     60    254 
 TAB_3  TAB_4  TAB_5  TAB_6 
   804   1278     24    120

In [ ]:
# Select only wildebeest predictions
preds <- preds[which(preds$species == "Wildebeest"),]

Identify the top 5 percent of bounding boxes highest in the image for each date download folder

In [ ]:
#Set up ojects
FOV_background <- data.frame("camera.no" = NA, "datedownload" = NA, "folder" = NA,
                  "position" = NA, "species" = NA, "species_length" = NA, "pixel_width" = NA, 
                  "pixel_height" = NA)
temp <- FOV_background
cameras <- as.character(unique(preds$camera.no))


In [ ]:
for(i in 1:length(cameras)){
  xx <- preds[which(preds$camera.no == cameras[i]),]
  datedownload <- unique(xx$datedownload)
  for(j in 1:length(datedownload)){
    yy <- xx[which(xx$datedownload == datedownload[j]),]
    yy <- yy[order(yy$y_pixel),]
    yy$id <- 1:length(yy$filename)
    yy$percentile <- NA
    for (k in 1:length(yy$filename)){
      yy$percentile[k] <- k/max(yy$id)*100
    }
    y_95 <- yy$height_pixel[which(yy$percentile >= 95)]
    temp$camera.no <- cameras[i]
    temp$datedownload <- datedownload[j]
    temp$position <- "background"
    temp$species <- "wildebeest"
    temp$species_length <- 2.050
    temp$pixel_height <- mean(y_95)
    FOV_background <- rbind(FOV_background, temp)
  }
}

FOV_background <- FOV_background[2:length(FOV_background$camera.no),]


Identify the top 5 percent of bounding boxes highest in the image for each month camera trap was in operation

In [ ]:
# Calculation for each month
#preds$date <- as.Date(preds$date, format = "%Y-%m-%d")
#preds$MY_date <- format(preds$date, "%m-%Y")
#preds$MY_date <- as.factor(preds$MY_date)

#FOV_background <- data.frame("camera.no" = NA, "date" = NA,
#                  "position" = NA, "species" = NA, "species_length" = NA, "pixel_width" = NA, 
#                  "pixel_height" = NA)
#temp <- FOV_background

#cameras <- as.character(unique(preds$camera.no))

In [ ]:
#for(i in 1:length(cameras)){
#  xx <- preds[which(preds$camera.no == cameras[i]),]
#  month <- unique(xx$MY_date)
#  for(j in 1:length(month)){
#    yy <- xx[which(xx$MY_date == month[j]),]
#    yy <- yy[order(yy$y_pixel),]
#    yy$id <- 1:length(yy$filename)
#    yy$percentile <- NA
#    for (k in 1:length(yy$filename)){
#      yy$percentile[k] <- k/max(yy$id)*100
#    }
#    y_90 <- yy$height_pixel[which(yy$percentile >= 90)]
#    temp$camera.no <- cameras[i]
#    temp$date <- month[j]
#    temp$position <- "background"
#    temp$species <- "wildebeest"
#    temp$species_length <- 2.050
#    temp$pixel_height <- mean(y_90)
#    FOV_background <- rbind(FOV_background, temp)
#  }
#}

#FOV_background <- FOV_background[2:length(FOV_background$camera.no),]

Calculate Area from Wildebeest pixel height

In [ ]:
# WB Height : 1.21m 
FOV_background$pixel_width <- FOV_background$pixel_height * 2.05 / 1.21


In [ ]:
# width of image based on file size
FOV_background$image_width <- NA
for (i in 1:length(FOV_background$camera.no)){
  xx <- All_Images[which(All_Images$camera.no == FOV_background$camera.no[i]),]
  yy <- xx[which(xx$datedownload == FOV_background$datedownload[i]),]
  FOV_background$image_width[i] <- as.numeric(substr(yy$resolution[1], 1, 4))
}

In [ ]:
# Calculate width of area in the foreground and background of each image
for (i in 1:length(FOV_background$camera.no)){
  FOV_background$fov_width[i] <- FOV_background$image_width[i] * FOV_background$species_length[i] / FOV_background$pixel_width[i]
}


In [ ]:
# Calculate distance from each example animal to the camera
for (i in 1:length(FOV_background$camera.no)){
  FOV_background$dist_camera[i] <- (FOV_background$fov_width[i]/2)/tan(0.3927)
}

In [ ]:
# Calculate area within the the field of view and between the animal and the camear
for (i in 1:length(FOV_background$camera.no)){
  FOV_background$AOV[i] <- 0.5 * FOV_background$fov_width[i] * FOV_background$dist_camera[i]
}


In [ ]:
# Change area from m2 to km2
FOV_background$AOV_km2 <- FOV_background$AOV/1000000
names(FOV_background)[12] <- "AOV_m2" 

In [ ]:
#Export Calculation Dataframe
write.csv(FOV_background, "AI_AreaOfView.csv", row.names = F)

In [ ]:
FOV_background

,camera.no,datedownload,folder,position,species,species_length,pixel_width,pixel_height,image_width,fov_width,dist_camera,AOV_m2,AOV_km2
,<chr>,<int>,<lgl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,KILE_5,20170126,NA,background,wildebeest,2.05,311.16486,183.663158,2592,17.076479,20.613080,175.99941,1.759994e-04
3,KILE_6,20170126,NA,background,wildebeest,2.05,40.87630,24.126984,2592,129.992211,156.914072,10198.80360,1.019880e-02
4,KUKA_1,20170612,NA,background,wildebeest,2.05,56.77082,33.508631,2592,93.597378,112.981737,5287.39719,5.287397e-03
5,KUKA_2,20170127,NA,background,wildebeest,2.05,41.46889,24.476757,2592,128.134621,154.671768,9909.40422,9.909404e-03
6,KUKA_2,20170824,NA,background,wildebeest,2.05,62.02840,36.611887,2592,85.663982,103.405305,4429.05511,4.429055e-03
7,KUKA_3,20170127,NA,background,wildebeest,2.05,47.04492,27.767975,2592,112.947380,136.339194,7699.57739,7.699577e-03
8,KUKA_3,20170824,NA,background,wildebeest,2.05,29.58139,17.460237,2592,179.626424,216.827712,19473.99325,1.947399e-02
9,KUKA_4,20170127,NA,background,wildebeest,2.05,39.85145,23.522073,2592,133.335188,160.949392,10730.10868,1.073011e-02
10,KUKA_4,20170612,NA,background,wildebeest,2.05,430.83306,254.296584,2592,12.333315,14.887590,91.80667,9.180667e-05
